In [10]:
import pandas as pd
from google import genai
from google.genai import types
import os

In [11]:
client = genai.Client(
    vertexai=True,
    project="gen-lang-client-0000496465",
    location="us-central1"
)

In [12]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Project_Tibame_Capstone\Project\flask_app\LLM4report\gemini2.0.json'

In [13]:
def csv_read_with_meanings(file_path="C:\\Project_Tibame_Capstone\\Project\\flask_app\\LLM4report\\finvision_tables_and_columns.csv"):
    """
    讀取 CSV 文件，提取英文欄位名稱及對應的中文意義，以及表名稱信息。

    Args:
        file_path (str): CSV 文件的路徑。

    Returns:
        dict: 表名稱到欄位對應字典，包含英文欄位名稱與中文意義。
    """
    df = pd.read_csv(file_path, encoding='utf-8')
    required_columns = ['table_name', 'table_name_cn', 'column_name', 'column_name_cn']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"CSV 文件必須包含這些欄位: {required_columns}")
    table_meanings = {}
    for table_name, group in df.groupby('table_name'):
        table_meanings[table_name] = dict(zip(group['column_name'], group['column_name_cn']))
    return table_meanings

In [14]:
def load_data_from_variable(data) -> pd.DataFrame:
    """
    Load data from a variable and convert it into a DataFrame.

    Args:
        data (list): JSON-like list of dictionaries.

    Returns:
        pd.DataFrame: Converted DataFrame.
    """
    try:
        return pd.DataFrame(data)
    except Exception as e:
        raise ValueError(f"Error processing data: {e}")


In [15]:
def generate_summary(data: pd.DataFrame):
    """
    生成數據描述性統計。

    Args:
        data (pd.DataFrame): 輸入數據。

    Returns:
        pd.DataFrame: 描述性統計。
    """
    try:
        if not isinstance(data, pd.DataFrame):
            raise ValueError("輸入數據必須是 Pandas DataFrame！")
        return data.describe()
    except Exception as e:
        raise Exception(f"生成描述性統計時發生錯誤：{e}")


In [16]:
def construct_prompt(user_input: str, summary_stats: pd.DataFrame) -> tuple:
    """
    構建系統與用戶提示。

    Args:
        user_input (str): 用戶查詢。
        summary_stats (pd.DataFrame): 描述性統計。

    Returns:
        tuple: (系統提示, 用戶提示)
    """
    table_meanings = csv_read_with_meanings()
    system_prompt = "以下是公司內部數據欄位及意義的詳細說明：\n"
    for table_name, column_map in table_meanings.items():
        system_prompt += f"表名稱: {table_name}\n"
        for column_name, column_meaning in column_map.items():
            system_prompt += f"- {column_name}: {column_meaning}\n"
    system_prompt += (
        "角色：你是一位專業數據科學家，負責從公司數據中提取洞察並撰寫報告。\n"
        "請根據以下數據進行分析並回答問題。\n"
        "分析結果須包含每個欄位的分析報告。\n"
    )
    user_prompt = (
        f"數據描述性統計如下：\n{summary_stats.to_string()}\n\n"
        f"請根據以上數據分析回答以下問題：\n{user_input}"
    )
    return system_prompt, user_prompt


In [17]:
def generate_report(data: list, user_input: str) -> str:
    """
    基於自然語言查詢生成分析報告。

    Args:
        data (list): 數據內容或數據文件的路徑。
        user_input (str): 用戶的自然語言查詢。

    Returns:
        str: 分析報告或錯誤信息。
    """
    try:
        # 加載數據
        data_df = load_data_from_variable(data)
        # 生成描述性統計
        summary_stats = generate_summary(data_df)
        # 構建 GPT 模型請求的提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)
        # 配置模型參數
        model = "gemini-1.5-pro"
        generation_config = types.GenerateContentConfig(
            max_output_tokens=8000,
            temperature=0.01,
            response_modalities=["TEXT"]
        )
        # 構建生成內容的請求
        contents = [
            types.Content(parts=[
                types.Part.from_text(system_prompt + user_prompt)
            ], role="user")
        ]
        # 調用模型生成內容
        response = client.models.generate_content(
            model=model,
            contents=contents,
            config=generation_config
        )
        return response.text
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"

In [18]:
if __name__ == "__main__":
    # 示例數據
    data =[
            {"seasons": "2022Q2", "TEQ": 259303805, "NICFI": -275900000, "FC": 2904000},
            {"seasons": "2022Q1", "TEQ": 259317675, "NICFI": -288100000, "FC": 2141000},
            {"seasons": "2021Q4", "TEQ": 259303805, "NICFI": -245300000, "FC": 2174000},
            {"seasons": "2021Q3", "TEQ": 259303805, "NICFI": -177000000, "FC": 1334000},
            {"seasons": "2021Q2", "TEQ": 259303805, "NICFI": -169800000, "FC": 1134000},
            {"seasons": "2021Q1", "TEQ": 259303805, "NICFI": -244300000, "FC": 772000},
            {"seasons": "2020Q4", "TEQ": 259303805, "NICFI": -90960000, "FC": 769000},
            {"seasons": "2020Q3", "TEQ": 259303805, "NICFI": -96080000, "FC": 373000},
            {"seasons": "2020Q2", "TEQ": 259303805, "NICFI": -129800000, "FC": 409000},
            {"seasons": "2020Q1", "TEQ": 259303805, "NICFI": -189000000, "FC": 531000},
            {"seasons": "2023Q4", "TEQ": 259320710, "NICFI": -132300000, "FC": 2919000},
            {"seasons": "2023Q3", "TEQ": 259320710, "NICFI": -242200000, "FC": 3112000},
            {"seasons": "2023Q2", "TEQ": 259320710, "NICFI": -259300000, "FC": 3005000},
            {"seasons": "2023Q1", "TEQ": 259320710, "NICFI": -272200000, "FC": 2964000},
            {"seasons": "2022Q4", "TEQ": 259303805, "NICFI": -342500000, "FC": 3335000},
            {"seasons": "2022Q3", "TEQ": 259303805, "NICFI": -284400000, "FC": 3370000}
        ]

    user_input = (
    """       
            "請分析財務數據並描述各季度的財務指標，包含欄位意義及異常分析。\n"
            "1欄位解釋：解釋欄位的意義。\n"
            "2趨勢分析：各指標的變化趨勢。\n"
            "3異常檢測：指出不尋常的變化並分析原因。\n"
            "4綜合評估：總結財務狀況的風險與優勢。\n"
            "不須最後的注意事項"         
     """
    )
    try:
        report = generate_report(data, user_input)
        print("生成的報告：")
        print(report)
    except Exception as e:
        print(f"程序運行失敗：{e}")


生成的報告：
## 財務數據分析報告

**數據期間:**  因資料提供不包含時間區間，以下分析將以 16 個季度為基準進行分析。

**分析欄位:** TEQ (股本合計), NICFI (投資活動之淨現金流入(出)), FC (財務成本)

### 一、欄位解釋

1. **TEQ (股本合計):**  代表公司所有者投入的資本總額，以及公司歷年累積的盈餘。股本合計越高，代表公司的自有資金越充足，財務風險相對較低。
2. **NICFI (投資活動之淨現金流入(出)):**  反映公司在投資活動 (如購買或出售固定資產、長期投資等) 中的現金流變化。正值表示公司投資活動產生淨現金流入，負值則表示淨現金流出。
3. **FC (財務成本):**  公司在籌措和使用資金過程中產生的費用，例如利息費用、匯兌損益等。財務成本越高，代表公司的資金使用效率越低，獲利能力會受到影響。

### 二、趨勢分析

1. **TEQ (股本合計):**  從提供的數據來看，TEQ 在 16 個季度內幾乎沒有變化，維持在 2.59 億左右。這表示公司在這段期間沒有增資或減資的動作。
2. **NICFI (投資活動之淨現金流入(出)):**  NICFI 的平均值為 -2.15 億，且呈現較大的標準差 (7.56 億)，顯示公司投資活動的現金流波動較大。 
3. **FC (財務成本):**  FC 的平均值為 195 萬，標準差為 115 萬，顯示公司財務成本相對穩定，但需進一步分析其組成項目才能更精確地評估。

### 三、異常檢測

1. **TEQ (股本合計):**  TEQ 在 16 個季度內幾乎沒有變化，這點值得關注。一般來說，公司會根據經營狀況進行增資或減資，以調整資本結構。TEQ 長期維持不變，可能表示公司缺乏明確的發展規劃，或存在其他未公開的財務操作。
2. **NICFI (投資活動之淨現金流入(出)):**  NICFI 的數據波動較大，需要進一步分析每個季度的具體投資項目，才能判斷是否存在異常的現金流入或流出。例如，單一季度的大額投資 (如併購) 或出售資產都可能造成 NICFI 的劇烈變化。
3. **FC (財務成本):**  FC 的數據相對穩定，但仍需留意是否存在隱藏的風險。例如，公司可能利用短期借款來償還長期負債，導致短期債務增加，進而影響未來的財